# Journal Club May 3rd 2024
This is an example notebook where I test how important correct labels are for few shot learning with GPT3.5. This was inspired by the journal club paper.

In this first section, I think I am prompting the model in the "right" way. For this chat model, I provide the instructions in the system section at the top, then the examples as conversations back and forth between a user and the model. Then I end with the query text as a user prompt. Hopefully this properly demonstrates the format of the task. 

Below, under OLD, I tried a different format where the instruction and all examples were fed in as a single user prompt. I feel like this is a less good way to prompt the model, however it seemed to perform maybe slightly better. Overall, I'm not totally sure what the right way is to prompt. 

In [6]:
from openai import OpenAI

client = OpenAI()

In [7]:
def gpt(examples, query):
    messages=[
            {"role": "system", "content": "You are a helpful assistant. Provide a label indicating the sentiment of the user's text. Respond with only the label."},
        ]
    for example in examples:
        messages.append({"role": "user",   "content": example[0]})
        messages.append({"role": "assistant", "content": example[1]})

    messages.append({"role": "user", "content": query})

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    return response.choices[0].message.content

A simple example of correct vs. incorrect labels.

In [21]:
query = "I like dogs."

examples = [
    ("I like cats.","positive"),
    ("I dislike birds.","negative"),
]
print(gpt(examples, query))

examples = [
    ("I like cats.","negative"),
    ("I dislike birds.","positive"),
]
print(gpt(examples, query))

positive
positive


A still pretty simple example of correct vs. incorrect labels.

In [20]:
query = "I think that birds are really cool."

examples = [
    ("I want to get a cat someday.","positive"),
    ("Dogs are way too loud for me.","negative"),
]
print(gpt(examples, query))

examples = [
    ("I want to get a cat someday.","negative"),
    ("Dogs are way too loud for me.","positive"),
]
print(gpt(examples, query))

positive
positive


Demonstrating labels. If I don't specify the neutral label in any examples, can the model still label the query as neutral? How much does random versus correct labels influence this? 

In [19]:
def run_five_times(examples, query):
    lst = []
    for _ in range(5):
        lst.append(gpt(examples, query))
    lst.sort()
    return lst

query = "Although I don't want one as a pet, birds aren't the worst."

examples = [
    ("I want to get a cat someday.","positive"),
    ("Dogs are way too loud for me.","negative"),
    ("Aren't peacocks really pretty?","positive"),
    ("I think that lions are terrifying.","negative"),
]
print(run_five_times(examples, query))

examples = [
    ("I want to get a cat someday.","positive"),
    ("Dogs are way too loud for me.","positive"),
    ("Aren't peacocks really pretty?","positive"),
    ("I think that lions are terrifying.","positive"),
]
print(run_five_times(examples, query))

examples = [
    ("I want to get a cat someday.","neutral"),
    ("Dogs are way too loud for me.","positive"),
    ("Aren't peacocks really pretty?","neutral"),
    ("I think that lions are terrifying.","neutral"),
]
print(run_five_times(examples, query))

['neutral', 'neutral', 'neutral', 'neutral', 'neutral']
['neutral', 'positive', 'positive', 'positive', 'positive']
['neutral', 'neutral', 'neutral', 'neutral', 'positive']


# OLD

I think the above formatting is more correct than the below formatting, but I am not totally sure. 

In [ ]:
def gpt(text):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": text},
        ]
    )
    
    return response.choices[0].message.content

In [ ]:
text = """Provide a label indicating the sentiment for the following text. Respond with only the label. 
Text: I like cats.\nLabel: positive
Text: I dislike birds.\nLabel: negative
Text: I like dogs.\nLabel:"""

print(gpt(text))

text = """Provide a label indicating the sentiment for the following text. Respond with only the label. 
Text: I like cats.\nLabel: negative
Text: I dislike birds.\nLabel: positive
Text: I like dogs.\nLabel:"""

print(gpt(text))

In [ ]:
text = """Provide a label indicating the sentiment for the following text. Respond with only the label. 
Text: I want to get a cat someday.\nLabel: positive
Text: Dogs are way too loud for me.\nLabel: negative
Text: I think that birds are really cool.\nLabel:"""

print(gpt(text))

text = """Provide a label indicating the sentiment for the following text. Respond with only the label. 
Text: I want to get a cat someday.\nLabel: negative
Text: Dogs are way too loud for me.\nLabel: positive
Text: I think that birds are really cool.\nLabel:"""

print(gpt(text))

In [ ]:
def run_five_times(text):
    lst = []
    for _ in range(5):
        lst.append(gpt(text))
    lst.sort()
    return lst

text = """Provide a label indicating the sentiment for the following text. Respond with only the label. 
Text: I want to get a cat someday.\nLabel: positive
Text: Dogs are way too loud for me.\nLabel: negative
Text: Aren't peacocks really pretty?\nLabel: positive
Text: I think that lions are terrifying.\nLabel: negative
Text: Although I don't want one as a pet, birds are cool.\nLabel:"""

print(run_five_times(text))

text = """Provide a label indicating the sentiment for the following text. Respond with only the label. 
Text: I want to get a cat someday.\nLabel: positive
Text: Dogs are way too loud for me.\nLabel: positive
Text: Aren't peacocks really pretty?\nLabel: positive
Text: I think that lions are terrifying.\nLabel: positive
Text: Although I don't want one as a pet, birds are cool.\nLabel:"""

print(run_five_times(text))

text = """Provide a label indicating the sentiment for the following text. Respond with only the label. 
Text: I want to get a cat someday.\nLabel: negative
Text: Dogs are way too loud for me.\nLabel: neutral
Text: Aren't peacocks really pretty?\nLabel: neutral
Text: I think that lions are terrifying.\nLabel: negative
Text: Although I don't want one as a pet, birds are cool.\nLabel:"""

print(run_five_times(text))